In [1]:
import foyer
import hoomd
from hoomd.data import make_snapshot
import mbuild as mb
from mbuild.formats.hoomd_simulation import create_hoomd_simulation
import numpy as np

from planckton.utils.rigid import connect_rings

/Users/jenny/miniconda3/envs/planckton/lib/python3.7/site-packages/mdtraj/core/trajectory.py:32: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [2]:
bz_str = "c1ccccc1"
bz = mb.load(bz_str, smiles=True, ignore_box_warn=True)
name = "bz"
bz.name = name
#bz.visualize().show()

In [3]:
#npt_str = "c1ccc2ccccc2c1"
#npt = mb.load(npt_str, smiles=True, ignore_box_warn=True)
#npt.visualize().show()

In [4]:
#py_str = "c1cc2cccc3ccc4cccc1c4c32"
#py = mb.load(py_str, smiles=True, ignore_box_warn=True)
#py.visualize().show()

In [5]:
box = mb.Box([5,5,5])
system = mb.fill_box(bz, n_compounds=10, box=box)
#system.visualize().show()

In [6]:
gaff = foyer.forcefields.load_GAFF()
pmd_system = system.to_parmed(residues=[name])
typed_system = gaff.apply(system)
#print(set([atom.type for atom in typed_system.atoms]))

/Users/jenny/miniconda3/envs/planckton/lib/python3.7/site-packages/foyer/forcefield.py:449: UserWarning: No force field version number found in force field XML file.
  'No force field version number found in force field XML file.'
/Users/jenny/miniconda3/envs/planckton/lib/python3.7/site-packages/foyer/forcefield.py:461: UserWarning: No force field name found in force field XML file.
  'No force field name found in force field XML file.'
/Users/jenny/miniconda3/envs/planckton/lib/python3.7/site-packages/foyer/validator.py:132: ValidationWarning: You have empty smart definition(s)
  warn("You have empty smart definition(s)", ValidationWarning)


On feat/rigid branch of my fork of mbuild, an initial snapshot can be passed in:

```
create_hoomd_simulation(structure, ref_distance=1.0, ref_mass=1.0, ref_energy=1.0, r_cut=1.2, auto_scale=False, snapshot_kwargs={}, pppm_kwargs={'Nx': 8, 'Ny': 8, 'Nz': 8, 'order': 4}, init_snap=None)
```

First, make a snapshot with 10 rigid particles--one for each benzene ring:

In [7]:
sim = hoomd.context.SimulationContext()

with sim:
    hoomd.context.initialize("")
    init_snap = make_snapshot(N=10, particle_types=["_R"], box=hoomd.data.boxdim(L=10))

HOOMD-blue 2.9.3 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 10/17/2020
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, J Glaser, and S C Glotzer. "HOOMD-blue: A Python package for
  high-performance molecular dynamics and hard particle Monte Carlo
  simulations", Computational Materials Science 173 (2020) 109363
-----
HOOMD-blue is running on the CPU


In [8]:
with sim:
    hoomd_objects, ref_values = create_hoomd_simulation(
        typed_system, auto_scale=True, init_snap=init_snap
    )
    snap = hoomd_objects[0]

notice(2): Group "all" created containing 130 particles
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 10
notice(2): Particles with 3 exclusions             : 60
notice(2): Particles with 7 exclusions             : 60
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
Processing LJ and QQ
notice(2): Group "charged" created containing 0 particles
No charged groups found, ignoring electrostatics
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Processing harmonic angles
Processing periodic torsions
HOOMD SimulationContext updated from ParmEd Structure


Want to do something with [ring detection](https://openbabel.org/wiki/Ring_detection) where rings are automatically converted to rigid bodies.

- convert to pybel mol
- use smarts matching to find rings
- make rigid

[SSSR documentation](http://openbabel.org/dev-api/classOpenBabel_1_1OBRing.shtml#_details)

In [9]:
system_mol = system.to_pybel()
rings = sorted(connect_rings(system_mol), key=lambda x: x[0])

#print(*rings, sep="\n")
#print(len(rings))

Now let's move the rigid body centers to the center of the ring and set the body IDs

In [10]:
for i,ring in enumerate(rings):
    inds = ring + len(rings)
    snap.particles.position[i] = np.mean(snap.particles.position[inds], axis=0)
    snap.particles.body[i] = i
    snap.particles.body[inds] = i * np.ones(len(ring))

In [13]:
print(snap.particles.body)
#print(snap.particles.position)

[         0          1          2          3          4          5
          6          7          8          9          0          0
          0          0          0          0 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295          1          1
          1          1          1          1 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295          2          2
          2          2          2          2 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295          3          3
          3          3          3          3 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295          4          4
          4          4          4          4 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295          5          5
          5          5          5          5 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295          6          6
          6          6          6          6 4294967295 429496

In [12]:
with sim:
    rigid = hoomd.md.constrain.rigid()
    
    r_pos = snap.particles.position[0]
    const_pos = snap.particles.position[rings[0]+len(rings)]
    const_pos -= r_pos
    #print(r_pos,const_pos)
    
    const_types = [snap.particles.types[i] for i in snap.particles.typeid[rings[0]+len(rings)]]
    #print(const_types)
    
    rigid.set_param("_R", types=const_types, positions=[tuple(i) for i in const_pos])
    rigid.validate_bodies()
    
    centers = hoomd.group.rigid()
    nonrigid = hoomd.group.nonrigid()

    hoomd.md.integrate.langevin(group=centers, kT=1.0, seed=42);
    hoomd.md.integrate.langevin(group=nonrigid, kT=1.0, seed=42);
    hoomd.run(1e4)

**ERROR**: constrain.rigid(): Central particles must have a body tag identical to their contiguous tag.


[-1.4753485 -2.2619445 -3.6145627] [[-0.03797805  0.3224213  -0.25076008]
 [ 0.12618876 -0.02723575 -0.3895805 ]
 [ 0.15876317 -0.35131502 -0.13995862]
 [ 0.03850293 -0.32225466  0.25089145]
 [-0.12214649  0.02861571  0.39078164]
 [-0.16333055  0.3497684   0.13862634]]
['ca', 'ca', 'ca', 'ca', 'ca', 'ca']


RuntimeError: Error validating rigid bodies


In [16]:
help(rigid.set_param)

Help on method set_param in module hoomd.md.constrain:

set_param(type_name, types, positions, orientations=None, charges=None, diameters=None) method of hoomd.md.constrain.rigid instance
    Set constituent particle types and coordinates for a rigid body.
    
    Args:
        type_name (str): The type of the central particle
        types (list): List of types of constituent particles
        positions (list): List of relative positions of constituent particles
        orientations (list): List of orientations of constituent particles (**optional**)
        charge (list): List of charges of constituent particles (**optional**)
        diameters (list): List of diameters of constituent particles (**optional**)
    
    .. caution::
        The constituent particle type must be exist.
        If it does not exist, it can be created on the fly using
        ``system.particles.types.add('A_const')`` (see :py:mod:`hoomd.data`).
    
    Example::
    
        rigid = constrain.rigid()
  